In [ ]:
# Suppress TensorFlow warnings for cleaner output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN custom operations

import warnings
warnings.filterwarnings('ignore')

print("✓ Environment configured - warnings suppressed")

# 🎭 Facial Expression Recognition (FER) - Kaggle Training

This notebook trains a **MobileNetV2-based FER classifier** for the Robotics Final Project.

## 📋 Project: Facial Expression Follower Robot

**Goal:** Classify facial expressions into 5 emotions for robot control
- **Happy** → Robot moves forward
- **Sad** → Robot turns right  
- **Angry** → Robot moves backward
- **Surprised** → Robot turns left
- **Neutral** → Robot stops

## 🔧 Setup Instructions

### **On Kaggle:**
1. **Enable GPU Accelerator**
   - Settings → Accelerator → **GPU T4 x2** (recommended)
   - Or **GPU P100** for faster training
   
2. **Add FER2013 Dataset**
   - Add Data → Search "FER2013"
   - Use: `msambare/fer2013` (organized folders)
   
3. **Internet Access**
   - Enable for downloading MobileNetV2 weights

### **After Training:**
Download these files to your local project:
- `fer_classifier.h5`
- `fer_classifier_fp16.tflite`
- `training_history.png`
- `confusion_matrix.png`

## 1️⃣ Setup Environment & Check GPU

In [ ]:
import tensorflow as tf
import numpy as np
import os

# Check GPU availability
print("="*60)
print("GPU/TPU CHECK")
print("="*60)
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

# Get GPU details
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(f"\n✓ GPU Detected: {gpu}")
        # Get GPU memory info
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("  Memory growth enabled")
        except:
            pass
else:
    print("\n⚠ No GPU found - training will be slower on CPU")

print("="*60)

## 2️⃣ Import Required Libraries

In [ ]:
# Deep Learning
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Data Processing
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")

## 3️⃣ Configuration & Dataset Path

In [ ]:
# ============== CONFIGURATION ==============
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4
NUM_CLASSES = 5

# Class names for 5 emotions (ONLY direct matches from FER2013)
CLASS_NAMES = ['angry', 'happy', 'neutral', 'sad', 'surprised']

# Original FER2013 class mapping (0-6):
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
# We will ONLY use: Angry(0), Happy(3), Sad(4), Surprise(5), Neutral(6)
# EXCLUDED: Disgust(1), Fear(2) - dropped to avoid confusion

# Kaggle dataset path (adjust if needed)
DATA_DIR = '/kaggle/input/fer2013/fer2013'

# Output directory
OUTPUT_DIR = '/kaggle/working'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"Classes: {CLASS_NAMES}")
print(f"⚠️  EXCLUDING: Disgust and Fear (poor quality/confusion)")
print(f"Dataset Path: {DATA_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print("="*60)

## 4️⃣ Load and Explore Dataset

In [ ]:
# Check if dataset exists
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

print("Checking dataset structure...")
print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"Train exists: {os.path.exists(train_dir)}")
print(f"Test exists: {os.path.exists(test_dir)}")

# List available classes
if os.path.exists(train_dir):
    available_classes = sorted(os.listdir(train_dir))
    print(f"\nAvailable classes in dataset: {available_classes}")
    
    # Count samples per class
    print("\nClass distribution (Training set):")
    for class_name in available_classes:
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path):
            count = len(os.listdir(class_path))
            print(f"  {class_name}: {count} images")
else:
    print("⚠ Dataset not found! Make sure you've added the FER2013 dataset in Kaggle.")
    print("  Go to: Add Data → Search 'FER2013' → Add 'msambare/fer2013'")

### Visualize Sample Images

In [ ]:
import cv2
from PIL import Image

# Display sample images from each class (only the 5 we're using)
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

CLASSES_TO_VISUALIZE = ['angry', 'happy', 'neutral', 'sad', 'surprised']

for idx, class_name in enumerate(CLASSES_TO_VISUALIZE):
    class_path = os.path.join(train_dir, class_name)
    if os.path.exists(class_path):
        # Get first image
        img_files = os.listdir(class_path)
        if img_files:
            img_path = os.path.join(class_path, img_files[0])
            img = Image.open(img_path)
            
            axes[idx].imshow(img, cmap='gray')
            axes[idx].set_title(f'{class_name.capitalize()}', fontsize=12, fontweight='bold')
            axes[idx].axis('off')
            
            # Show another sample
            if len(img_files) > 1:
                img_path2 = os.path.join(class_path, img_files[1])
                img2 = Image.open(img_path2)
                axes[idx + 5].imshow(img2, cmap='gray')
                axes[idx + 5].set_title(f'{class_name.capitalize()} (2)', fontsize=12)
                axes[idx + 5].axis('off')

plt.suptitle('Sample Images - 5 Direct-Match Classes Only (Excluding Fear & Disgust)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 5️⃣ Data Preprocessing & Augmentation

In [ ]:
# Training data: with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.1  # Use 10% for validation
)

# Test data: only rescale
test_datagen = ImageDataGenerator(rescale=1./255)

# IMPORTANT: Only use the 5 matching classes
# Exclude 'disgust' and 'fear' folders if they exist
CLASSES_TO_USE = ['angry', 'happy', 'neutral', 'sad', 'surprised']

# Create data generators with class filtering
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES_TO_USE,  # Only load these 5 classes
    subset='training',
    shuffle=True,
    color_mode='rgb'  # Convert grayscale to RGB for MobileNetV2
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES_TO_USE,  # Only load these 5 classes
    subset='validation',
    shuffle=False,
    color_mode='rgb'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES_TO_USE,  # Only load these 5 classes
    shuffle=False,
    color_mode='rgb'
)

print("="*60)
print("DATA GENERATORS CREATED")
print("="*60)
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")
print(f"✓ Using only 5 classes: {CLASSES_TO_USE}")
print(f"✗ Excluded: disgust, fear")
print("="*60)

## 6️⃣ Build MobileNetV2 FER Model

In [ ]:
def build_fer_model(num_classes=5, img_size=224):
    """
    Build MobileNetV2-based FER classifier.
    
    Architecture:
    - MobileNetV2 backbone (ImageNet pretrained, frozen initially)
    - Global Average Pooling
    - Dense(128) + ReLU + Dropout(0.3)
    - Dense(num_classes) + Softmax
    """
    # Load pretrained MobileNetV2 (without top classification layer)
    base_model = MobileNetV2(
        input_shape=(img_size, img_size, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Build model
    inputs = keras.Input(shape=(img_size, img_size, 3))
    
    # Preprocessing for MobileNetV2 (normalize to [-1, 1])
    x = keras.applications.mobilenet_v2.preprocess_input(inputs)
    
    # Extract features
    x = base_model(x, training=False)
    
    # Classifier head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu', name='fc1')(x)
    x = layers.Dropout(0.3, name='dropout')(x)
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
    
    model = keras.Model(inputs, outputs, name='FER_MobileNetV2')
    
    return model, base_model

# Build model
print("Building model...")
model, base_model = build_fer_model(NUM_CLASSES, IMG_SIZE)

# Display architecture
model.summary()

print(f"\n✓ Model built successfully!")
print(f"Total parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

## 7️⃣ Compile Model

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top2_acc')]
)

print("✓ Model compiled successfully")
print(f"Optimizer: Adam (lr={LEARNING_RATE})")
print(f"Loss: Categorical Crossentropy")
print(f"Metrics: Accuracy, Top-2 Accuracy")

## 8️⃣ Setup Training Callbacks

In [ ]:
callbacks = [
    # Save best model
    ModelCheckpoint(
        os.path.join(OUTPUT_DIR, 'fer_best.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1,
        mode='max'
    ),
    
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when stuck
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Log training history
    CSVLogger(
        os.path.join(OUTPUT_DIR, 'training_log.csv'),
        separator=',',
        append=False
    )
]

print("✓ Callbacks configured:")
print("  - ModelCheckpoint: Save best model")
print("  - EarlyStopping: Patience=10 epochs")
print("  - ReduceLROnPlateau: Factor=0.5, Patience=5")
print("  - CSVLogger: Log metrics to CSV")

## 9️⃣ Train Model (Frozen Base)

Training in two stages:
1. **Stage 1**: Train only the classifier head (base frozen) - Fast initial learning
2. **Stage 2**: Fine-tune top layers (unfreeze some base layers) - Better accuracy

In [ ]:
print("="*60)
print("STAGE 1: TRAINING WITH FROZEN BASE")
print("="*60)

# Train with frozen base
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Stage 1 training complete!")

## 🔟 Fine-Tune Model (Unfreeze Top Layers)

In [ ]:
print("="*60)
print("STAGE 2: FINE-TUNING (UNFREEZING TOP LAYERS)")
print("="*60)

# Unfreeze the base model
base_model.trainable = True

# Freeze early layers, unfreeze top layers
for layer in base_model.layers[:100]:
    layer.trainable = False

print(f"Total layers: {len(base_model.layers)}")
print(f"Trainable layers: {sum([l.trainable for l in base_model.layers])}")

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"✓ Model recompiled with lr={LEARNING_RATE/10}")

# Fine-tune for additional epochs
FINETUNE_EPOCHS = 20

history_ft = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=FINETUNE_EPOCHS,
    callbacks=[
        ModelCheckpoint(
            os.path.join(OUTPUT_DIR, 'fer_finetuned.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
    ],
    verbose=1
)

print("\n✓ Stage 2 fine-tuning complete!")

## 1️⃣1️⃣ Visualize Training History

In [ ]:
def plot_training_history(history, title_prefix='Stage 1'):
    """Plot training and validation metrics."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0].set_title(f'{title_prefix} - Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[1].set_title(f'{title_prefix} - Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot Stage 1
fig1 = plot_training_history(history, 'Stage 1: Frozen Base')
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history_stage1.png'), dpi=150, bbox_inches='tight')
plt.show()

# Plot Stage 2
fig2 = plot_training_history(history_ft, 'Stage 2: Fine-Tuning')
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history_stage2.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history plots saved")

## 1️⃣2️⃣ Evaluate on Test Set

In [ ]:
print("="*60)
print("EVALUATING ON TEST SET")
print("="*60)

# Get predictions
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Get class names in correct order
class_names_ordered = [k for k, v in sorted(test_generator.class_indices.items(), key=lambda x: x[1])]

# Classification report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_names_ordered, digits=4))

# Overall metrics
accuracy = np.mean(y_pred == y_true)
print(f"\n{'='*60}")
print(f"OVERALL TEST ACCURACY: {accuracy*100:.2f}%")
print(f"{'='*60}")

### Confusion Matrix

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names_ordered,
    yticklabels=class_names_ordered,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - FER Classifier', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=13, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

# Save
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✓ Confusion matrix saved")

## 1️⃣3️⃣ Save Final Model

In [ ]:
# Save final Keras model
final_model_path = os.path.join(OUTPUT_DIR, 'fer_classifier.h5')
model.save(final_model_path)
print(f"✓ Keras model saved: {final_model_path}")

# Get file size
size_mb = os.path.getsize(final_model_path) / (1024 * 1024)
print(f"  File size: {size_mb:.2f} MB")

## 1️⃣4️⃣ Export to TensorFlow Lite (for Raspberry Pi)

In [ ]:
print("="*60)
print("EXPORTING TO TENSORFLOW LITE")
print("="*60)

# Convert to TFLite with float16 quantization (smaller, faster)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

# Save TFLite model
tflite_path = os.path.join(OUTPUT_DIR, 'fer_classifier_fp16.tflite')
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
print(f"✓ TFLite model (FP16) saved: {tflite_path}")
print(f"  File size: {size_mb:.2f} MB")

# Also save full precision version
converter_full = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_full = converter_full.convert()

tflite_full_path = os.path.join(OUTPUT_DIR, 'fer_classifier.tflite')
with open(tflite_full_path, 'wb') as f:
    f.write(tflite_full)

size_full_mb = os.path.getsize(tflite_full_path) / (1024 * 1024)
print(f"✓ TFLite model (FP32) saved: {tflite_full_path}")
print(f"  File size: {size_full_mb:.2f} MB")

print("\n📌 Use FP16 version for Raspberry Pi deployment!")
print("="*60)

## 1️⃣5️⃣ Test TFLite Model Inference

In [ ]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite Model Details:")
print(f"  Input shape: {input_details[0]['shape']}")
print(f"  Input dtype: {input_details[0]['dtype']}")
print(f"  Output shape: {output_details[0]['shape']}")

# Test with random input
test_input = np.random.rand(1, IMG_SIZE, IMG_SIZE, 3).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

print(f"\n✓ TFLite inference test successful!")
print(f"  Output: {output[0]}")
print(f"  Predicted class: {CLASS_NAMES[np.argmax(output[0])]}")
print(f"  Confidence: {np.max(output[0]):.4f}")

## 📦 Summary & Download Instructions

### ✅ Training Complete!

**Model trained on 5 CLEAN classes (direct matches only):**
- ✅ Angry (FER2013 class 0)
- ✅ Happy (FER2013 class 3)
- ✅ Sad (FER2013 class 4)
- ✅ Surprised (FER2013 class 5)
- ✅ Neutral (FER2013 class 6)
- ✗ **EXCLUDED**: Fear & Disgust (confusing/rare classes)

**Files to Download:**
1. `fer_classifier.h5` - Full Keras model (for reference)
2. `fer_classifier_fp16.tflite` - **Main model for Raspberry Pi** 
3. `fer_classifier.tflite` - Full precision TFLite (backup)
4. `training_history_stage1.png` - Training plots (stage 1)
5. `training_history_stage2.png` - Training plots (stage 2)
6. `confusion_matrix.png` - Model evaluation
7. `training_log.csv` - Training metrics

### 📥 How to Download from Kaggle:

1. Look at the right sidebar → **Output** section
2. Click on each file → **Download**
3. Or click **"Download All"** to get everything

### 🚀 Next Steps on Your Local Machine:

```bash
# 1. Create fer_model folder (if not exists)
mkdir fer_model

# 2. Move downloaded files
# Place fer_classifier_fp16.tflite in: fer_model/
# Place other files for your report

# 3. Test full pipeline
python main.py
```

### 📊 Expected Performance:

- **Test Accuracy**: **65-75%** (better than 7-class or merged approach!)
- **Inference Speed on Pi 4**: ~50-100ms per face
- **Overall Pipeline FPS**: 10-15 FPS

### 🎯 Emotion → Robot Action:

| Emotion | Robot Action | Index |
|---------|--------------|-------|
| Angry 😠 | Backward ↓ | 0 |
| Happy 😊 | Forward ↑ | 1 |
| Neutral 😐 | Stop ■ | 2 |
| Sad 😢 | Turn Right → | 3 |
| Surprised 😮 | Turn Left ← | 4 |

**Note:** Class order is alphabetical: angry, happy, neutral, sad, surprised

---

**Great work! Your FER model is ready for deployment! 🎉**

**Why this approach is better:**
- ✅ Higher accuracy (no confusing classes)
- ✅ Better precision per class
- ✅ Cleaner training (no noisy labels)
- ✅ More reliable robot control